In [1]:
"""
Python script to scrape Friktion User Data from Bitquery GraphQL API.

"""

import json
import requests
import requests
import traceback
import time

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from os.path import exists

In [8]:
globalId = "mainnet_income_call_luna"

users = pd.read_csv("luna_call_addresses.csv")

shareprice = pd.read_json(f"../derived_timeseries/{globalId}_sharePricesByGlobalId.json")

shareprice[0]/=1000

In [9]:
res = {}

In [10]:
def calc_amount(addr):
    success = False
    while not success:
        try:
            query = """https://solana-stream-dev-ztbl.ue1-eks-0.prod-czff.zettablock.dev/graphql?query={userTransactions(userAddress:\"%s\"){Hash,Amount,Decimal,InstructionAction,InstructionIndex,InnerIndex,UserAddress,Timestamp,UnixTime,CurrencyName,CurrencyAddress,SenderAddress,SenderTokenMint,ReceiverAddress,UserAction,GlobalID,VaultAuthority,ShareTokenMint,DepositTokenSymbol,DepositTokenCoingeckoId}}"""
            x = pd.DataFrame(json.loads(requests.get(query%addr).content)["userTransactions"])

            success = True
        except Exception as e:
            print("Failed... sleeping for 5")
            time.sleep(5)
            continue
            
    cutoff = 1651802400
    if x.empty:
        return

    df = x.loc[x.UnixTime < cutoff].loc[x.GlobalID == globalId]

    df = df.sort_values("UnixTime")

    shareprice["timestamp"] = shareprice[0].astype('int')
    shareprice["sharePrice"] = shareprice[1]

    asimov = pd.merge_asof(df, shareprice,  left_on="UnixTime", right_on="timestamp", direction='forward')

    pos = asimov.query("UserAction=='Deposit'")
    pos = (pos.Amount/pos.sharePrice.fillna(1)/10**pos.Decimal).sum()

    pos2 = asimov.query("UserAction == 'CancelPendingWithdrawal'")
    if pos2.empty:
        pos2 = 0
    else:
        pos2 = (pos2.Amount/10**pos2.Decimal).sum()

    neg = asimov.query("UserAction=='Withdraw'")
    if neg.empty:
        neg = 0
    else:
        neg = (neg.Amount/10**neg.Decimal).sum()

    neg2 = asimov.query("UserAction=='CancelPendingDeposit'")
    if neg2.empty:
        neg2 = 0
    else:
        neg2= (neg2.Amount/neg2.sharePrice.fillna(1)/10**neg2.Decimal).sum()

    amount = pos+pos2-neg-neg2
    amount = amount if abs(amount) > 0.001 else 0
    print(addr, amount)
    res[addr] = amount

In [11]:
for addr in users.userAddress.unique():
    calc_amount(addr)
    time.sleep(0.5)

CQikhhQ9STuD1S3A45DGvUAtCdSs5pw2YwJ8Mk6U9Fs7 180.6459547686176
ETv2idiPjun5RReJNSLBd2RUMcYSBMqjFQxhVpLeDqkm 11.389008021627172
GZYHrKmMPQLrmVGw7PRny7tP9wQZnFCC5vYfuaQiVTdh 0
24CoH2cYsEPv3EpigCUjsqs26n1XeDrtHug4kwX7UZGD 89.30916431715201
8vRaeK5hp4T3n8vyzfeHkHaUYqsXRrWBiAh3DDBn9dqe 0
2TGaPLxbFVgp2T2UycxEexbifxGeKbQGDVfiy2xZnoMN 37.09273806905467
E3Jwp134uzpA2MyCadYmeqWAxK8zDBmcNXTvSRrVK7EN 11.768393815791308
BxC131fG5TgGE1Wek7WtU4cTJcHWAs5cTmEmnx5BmQus 3.3220402345281257
BRNYKBf4wSfV8iZRa64fmjaerVUJjE38zR1Jmb8kQFkE 9.466138087411348
HdaigHQuEdsR6LsXZbgT37ohcbs1ZxEtSR2cudpWzZ2S 0
43o5vw2xsdC5qghLzrCrEB9z8Xz1JWFQU524nwAjFyKo 0
FPcL3sFutPQcV4hSvwBrhz19oJ6MNeGsaSMSYP1o4Rfh 0
CTgwTfirY2dtLbEBKYFe3T9KmeDLtf6QrHmtuAMwDTsP 122.06814182254777
7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN 0
775ANAGx3kyFQV6YHvQXmfA7gB6DFkyojsQZXVdF4vwr 0.5113793686204104
E6JjHtYpJkZqaYXa2WDccS39wABSkvkjzhwhe28rVKeZ 0
7g2UxLyBrHMFpGHKi938Mr4zXp8zBKhV3KN8Gb6TATBT 9.60122877090325
7FY5mjWJC2pxgQBS6tyAXHYUJMdL6Rq9ejWLt

8ZcKyXhKU9hqCozAa1PbkQY7sfwfph1wnFDNrcU8zGgy 0
95FmZxcD3hwPFHSUys5s1EUxqSRhPxp64HFuuuRUhYH9 0
GEAuYyk6eLynGGv7DrJUn8PRtoiBxyWrPQ5PFCTYF5Pz 0
9DWREMQtcKKcZRzqBi4kWKp61TW9MhASCVE9biCX7tvD 22.756131498779155
4ybC74KTXPprWPDT6iKwmqHKktD6bEJ1oxLgozA1Lkrs 0
BobSdbnTKaJdPzdUGAX4C2hf1Y21axA8Tv7LhtwEhjWF 0
FztjfAS4WdEACb7QNDxXE1S5dtyWNfFvaKSCLrsxyRF1 0
BYPQLo7SnhMp2Y9geafwAYqMm8u8uHnqDkdQPV5YaF4Z 0.14684972140770075
5HRdzXsEd9wT1UueqEAtGaBmjHBzqw6rcQr1yCrR58ap 0.18476198816695444
6v77cA6fA4yrytwGUSuB9jfeoHM5ota6KMmULBkg1J2J 16.972215368321034
FiQgDST83RUgHbhG6iwL7t1zNngvEbUCoQVga4eW2eec 9.8451409880264
6rCkkRASBhePAs6YkbK9zh1kJaPre9jb7QRDt7kVcXMz 0
Cpg2mA1MetDk4ZacEvtAztTuzDMTjSy1rQBfEs16AEbn 0
C35WVe5uAj5UMSi2HEG1V5RqPkvwJ4uudxM4hixr5KdS 0
Fm17BofEun9K6xfLJ6ZrbR9773g6GZfJnBKHYkvK8GZi 0
8LSTxq6teYuBZ6PjjpLzLgTyx2tpR36cpZxT47cSVJs4 0.10077413468090751
C98xxhUuzsaVtPyyGbYnCJnheC3go4xC6SS6GQBZHdUs 0
DDuoHEtUXvj2HPYsqKitgu5Xb1KvRKMw9dVGjB6sgYbG 11.632740663970957
9e9cprGgqqqV9DaJ5ekt7b65irfxkFFFXDM

ByehdQv8djXcqv55qtU63fbpH26AqJExVLswaczVgzeo 1.1337174037504925
ACmQPUQVCYfv1eykgMsKLyQWtzqj6mP9CFxcP5zzMkBu 961.1254211321595
Cs2FzUwrfPyymEWB5G3WxySLyc6NYYGquaczyk24FwEb 1.3247953235317913
bxatWbxkiPhTN8rU3UcbjoTNNDDJATUA9cK3raCGP3Z 0.19700333000419884
9wiVvGNSmL5W92hruMru23G39Z4fxDmLdjGuQPormBqi 109.54349637787357
EDvWBmYG1fBFy4aGuo4kn42R9WeUAcxUU1T6GFjVcj4K 0.1190265448544507
335jrByeq8hoFoxcFQvew5aqnXhFX4m6oyKeFcErkVq3 0
Aw8JjQcTCQX7hNJwMDm7KzF7rzQ1tDYzqrDkUYZEBeVU 0
599pPebzJRxqXLcCCDcbucr7urGvH6t2egitrUGzXbLd 0
3ApbDZttLzq53RF7ugv1MHf4EZWapxeKD3nbnaNoVvU9 8.021789473525816
DX4JcwyMAzFe8gWo56BvJSdpLtbXzvfYDF9tdDBzSqna 0
5XgStSeDe22PVKojnuHaoboNGiJxgeszhLHsQYAgEBvs 0
73Z1WZL5JsbU5SVh43DCXhf3Dis1F2fx1FqihSm5zKhQ 0
1Mrfsd5xD9PyGWjCBJtcrBPkDirxUdqSTpmAouPUNfc 0
EGMpNSRTkfAeCKisyDhJqfwhzGTGsfREswsB5GhDPvTy 28.930566857278002
8KDV3yUf3jzzRUCCsTxgRBcwYyJbuLm1qaXJZfSbnKkc 0.7431228439224483
AwpGN89M7dZH7DR9LcoXHDnAWsLSa3c6FCneMqVgRVhr 1735.7693429510277
2diX1Sor7SBwQy683e78b2PrAQZxaKBYHw

CB1jtDXxwU7g9SvKmFFJNDAB6HEQpKbCYmWWzX5yiTba 0
J5nvCdysjMiYXMV6EtwT8TERjYQEtrbiwRzDfjjL3tNg 1.793746955525474
6fDS4Mh8bhgyVqn4xeDmxUMGfq2bgD3xUCUMiBVuPuVL 1.922250842264319
8MtYFYo2xJcKre46Bq5uGgeRgKkbyLBYddiCqkKJuNuN 361.9703871480198
DAPT6LmkBbvqp8MTJXFKM33B9z86vnMzLnTUu4fW6nrf 0
AGYuu3zsPrnvLQMGwADMq1Lz1vhLmaDjMXKAVrW9kQ5U 0
DAWYfjMzZjpTC5vDBXDN53HtdirKqmQ9FRwurPVQejRu 0.4819742472590264
7fVvdwZ9y4Hjj96d2zejnmucB9fSkgaogvKtqPbXJVsn 21.135165086376528
eiLMepBYM3VcWarQJC9fTnxotyMuUTbyMGJECedPEfu 0
6VHKRsV67Hx6zUWdQs8FVNFW7Qb2SBsEd6jqMRD5UVGf 0
E6gtYUmFLbfiEMyeHa9uQpoyS1bqH2Zi3SKYvTALkdvQ 57.55361040914951
5NaR7UXF7FgAHzYHcFcE9GyQ1bD2AxcL1d4JXfCjPZQk 0
34zpUSwzasQMfG86eDeyxfVuwvewR3SJub4BegWFY4nn 0.795812065883164
AUECYvjLKtHJsHrJWUXcyRbs6EXFhi5QNRjD6MpsmH5t 0
7iPoDrXP8hFAJgCheLV7AFHrgmAHj8Fgh1VTgZyeeJs6 0
FCUK1zE1tT3xzn2kBSAV9GYg9AdBgDFjimSuECXFtp4h 7.574187492061986
Eoh3YGNpVmj39KZkha93po7Yk4CMTbHSVYqBc41CvEj6 0
EkTuGJFG4upFR8XBEkFvsXb6s4SN18rS86fWjt7cDBGu 0
6j2AyyVXDAg64cawjTVhJMJdU

AQfV6JkvnoLZ5hBWc99bq6ZV7nr6bdbMHLLCWGty5dmc 0.8924929189927632
CWbmDZMCvwYvtxWWTRww3gQVBiFDJ7Jd6fZLb55Tn79B 0
5tHNyNQFvJRmXrVoj12S6mpXK8WAUzgM6JUALMb6TRuf 24.525536705271062
2G9Mdj5rrA5fybW6sb1q3SEoP5cfZBjgu6h8AhVnHfjS 43.554027263408884
EBsL5brszUrwnihzC9XPH6GxComHMFuGkBX8NXHdqQem 15.19398486726372
DbvdEbFzyjfcngH4A33sZANR8NSvMNLGEwCEKozmDyYt 0
EgzRJHeqbfYsCL4VBZCsq2ZEpk5M7jzWc4n2N53SyFKJ 47.73990209844567
7VxEFziAx7xoYNe5Gyatv7pZd7V5TgcdWhVF4Y7Qq5o 61.83258751727486
D3yaAXitJhiW2eVm21iySXQgh66vgnm3WJfM812Wek8p 0
EQBUDsacUvo86jPvTHms7fQa6i9Dej46PrYo3o6yrGKK 0
J3twGYQav3Von2PHPBv5FkWEB9gnGHWzGTS5AXDCipd8 0
5oEh3y9AcL2R5pZxfgTVtZfcTxkf19wytnawZJKpPwRV 0
9kotvvRV7uibWUdFzJKTJF7u2SzY66vRNKLhampgeux5 0
5LnP43oRSeRkPrTZCYxf1ndz3RDswStx9pFRrVww1Xyo 10.906564304945794
cWVBFCqXdiMJJkTHtTVkmz1Kqf8ChHEQfGCn3xBayto 0
BkNJtSq3dH1gzG3FMGkFpegdMYuUpDPxDdydTvJU3Md5 0
2zfkX9GfJHfDUH6MQDe67Qpqf61xaotvhwEpZd8JvpWK 0
7qxEXo4Y3jWLwXo7zg7LHEcRjKmvaw7pFNkDzc6JNDGw 0
FJ7CGdgWzSvD6mgHFrG6cE8zC3onSvKFLui8dQFa

BcMgwuZrorHzL3xnoKtaNMMZ41mqZ9tsuEdgMyqggzuc 870.5337887700888
ijbFSAVB6wGycXSxaHLdTD3CJheJqxj79DcVVCk17hD 0
ELJL2KGPWuHsUeGtdEJNp4YsrTGbcojyS5KhgqAVpwBh 944.2487085118543
F22thcgJMvXouai4zzcqnn7QM143mgittpK2y5GDPxzK 0
AGSt8UgvCsYP1BMZkrTkBL4ergLLZr5ovBZBuDZejzk2 0
ES2XtYGREZgTs9NsCZUCjWso1YQcouysa1Ak2hTfJEkY 12.09316611310315
GvtXZHZt5utPQ67ez3HLmbEtUXpspTi14y999aV7JKos 89.03478760048694
CL2JZC5qKtJQu2fisjCRb5N83EinjwahGAUYFdQAW4GN 375.73122232475265
9ZcdFE48RK3epAn2DGsH5QJBUz9ZZkJ92MirVbsuCkJV 0.9446695246276918
BkxPN6U1W7dSVvMxuVXuAt9NocpxLH2q47r2cwrJkZBU 0.9385954754018841
AZhjGXKAUwYsQZ6tszgt7JhVTkJg92i61FQ1Eh7QWQE8 0
DFKdVqgn3NS13s4wfMKSUXt86CEusGhVJ3YYEcDNpppR 0
AWsU5CcuMKLqVHCS4PuQqDXmtK8uVxDTun9zF4nGVNTH 0
EpcmWyd5agwHtzi6YisVcaQuB9hZFjTnEtZpxWGthLf4 0
ARGATMwBCdxy574dyREpZxVYWRexM53MGV9ximHq9Qjh 0.911412929135632
CsPYfWssqoyLqsVQWbdXViwoMDsH5eh91LntQuBuGzLa 19.22250842264319
BR3EKtEvi95xofZsV32YPkGeUcaLbXuHs2WZptefuWmF 0
2ukWCfsHwkXQZ5Yt4vL8412kNA9GTxyDNmRTyqAKBDkP 0
93rDhJb5

HTVLgwifE9xNRtKGEDPEeqLk1nVYZTnd5EnBXYKhLWb8 0
E2DAq6k6imt545hgsD2DxHgRe7ma9sqb29M85mKfgcc1 0
C2CtWcQpJ8ncG2PJmSWWM8mRhr7z1eGFF7A3fqAVYziT 0
vxJJrvNz8FR1QYg1v9JfSJXxPdfvrof8h6PkVbqwMYJ 1.922250842264319
3aofaB7D22bLWZfJN1ZWyej29ZuN2u2CDjgP91uddy1R 0
CkJ33M4Fi1NVwvJ9EXgTxNEzJGKG3DvCvZqaHnM2koFP 1.9429933104936545
thK2MxShYWG8siTLFmhsm5XdCknby6nwM6FbEDrx9F5 0
YPd29oLJ6s8MpT7ZaGasPA6k72wNLVUrcKBYZ8LniTv 0
4WoNVoeK7YY6y8X15btdTxHRQcgMDSHZM31ZQxq7v5oc 0
7hRTQr25zjb6KBNzmSNVvvS8B4V5fqwW6xQwSV14M7vG 1.6517359175698534
B7gJft11wxLLdXqKiYxv7C5SrmoxapRTBpHqUjvg5J7v 0.00222641326709823
CPPWWxxpnepyc7oy8KEXqMJeQSkCDksNwFBNhVexRhbQ 0
ADzfyniRKpi5ofjzNi2n9xEARvsTU9csC8o5K7zhkv5U 0.49072015194978874
CcRuy5mQwjHqR7omxWF6A7PH5UyxfeshUUzqk5mRBQt1 0
9kguvX5wfDtcycvYpgSsvbrKMF5WBTSpxNYov6RhtCA7 0
7B8Y5Vs89J4n6LxVAzBwNGo2gbxhkTdfauUHt2EHMLk8 0
4sbN39tGRx3HN4xtUdzmwpGSSAfMqgAPQ1iWiuam5FEy 0
HWxgoMt5aYqb66439pa5mBDKvZywBnB3LgazpJVTBvVM 5.8427371309344
2Dad6EnEWzrn5y2oekwyM83SB8CUUVAd8BGrDkJpkpmW 0
DoKNEBCGEi

AAxpQxY2r1G732B2D5m5EE4joJxmGisQuoWuDY5ZhdKH 0
6j1YjC11BLLwWBf627cxmimxN5NVHdjzR7Z8VKDqLCff 0
ywn8qUpuMwjCPWXDGRj1sUKKUYJrApKpmx2p3GrXbcb 38.27966610283854
8Sbwd3AzCT1Z5BdMvfuS76aDiYVwHd7UsVGouq479JkN 0
AKvwhqxnJHQg65BWMCCzAzEhkwMJJ6p22hw4yGvqTJeB 0
6rJzncczhAJTVvCtLfEdW33gat3CmT87yFEK36Y7wHZ9 0.15940457334561092
AG25ARitey2E2kUoBHSRnUSWLNPErPsvMyGqQCVfQHFG -3.8056821367288194
4hc6ghzCTnAbAETVpfPNAut95zdYGyJwysWiFuLope8D 0
ESShcBD34wXp5YtHFKhuC8gb3L4TwXpNcKBgpNFXA9kY 0
GesagsHCJWGmHvSWbtFiyRAF31yMkJobSSfjJxek953n 9.816098098582124
AtxzQ6EzB2fuortxq9WzgYTKeieR7cMuTPcdckWmrWUn 4.878685504330224
62VF2UZxppWDv3Ne52LDE8AhbMmqTByaBC7m2sfJJDbA -1.113193
7RVSWPMR4ZPP5A4M2GQHXuzgnUrEPV9742Y2TTWkHH9N -108.34488182114927
Ej8eoaJVVhgjRioafZ9G8paidP7Dpq5vpaaQUeuMLWKy 0
D5sNJz4aT1VPghmL2LGHUknenY59eBtVzWRDQK325LaN 0
7fay6fzfGtGXSYEEeh3S6WTxy8XQdYN1k6swQssBYPW9 0
HT8N3K1GWgF5DDNEfvmNLuN5A8ns8zoKqRUzHttU1q97 0.31167250278402747
5F5tK6z2m1g7GcUTk8de96UXcpDVEqWDYD4dZE6tf64C 14.417730230133436
GLLtHpmJ3S

xGxKvYxkMaKpYwx8iEee7skwgb5nh74PEcaBxUsaZfL 0.9359645523639399
A4wwN4KWphzUVE59AB7upCMuzHoRTwBssmtnWUiKknVe 0.6619149925128601
6mZnKPt3Mq4Sj8ADneAtxN2a3yGKceso7HNQK7dowHm2 3.324730823532893
6dkwmzF6Hbp4m4kHDXFsfLCL2i5PjHH1ZHy8mu1twLTX 0
62xWmC291kjpkYiXfDDzehSSqR4kZc4YGWKwSejiA8km 0
6ni6RE4Zjx3i3THtySXtsLrtQC23io1YR4Y8eC3ryp1c 98.94407207748696
4ChrPph2rCZ9bRqD6eo8xgeaKYMpypWyCDFB4aWUPqXp 40.33285339164553
CD9nFUXijjhLy92cMgWNdQ3R2RmR1aCLyXQWFUGWY9Gq 16.653754488287582
AwiWPGyo1M4vUBaMhqYoz8Z8Zj9UmyBt6igJYfvruv7r 0
HVwSVBTeMCUt8koznn2jRxFHyrn8oE37MemCjx2nN9Hv 10.723973908648297
HSSXHiV2SGJQjcZ8nAKSjyv2hNfDmw1tapDw18DL5tn1 0
Et2RvhY5Wg11otWSp491VR7LZoLS3DecfoZgk3unUnua 1.328635368259813
GZM4y4uBvmuKKWRA41vpGNGiAKRchLBknMSx23xdwUbJ 32.90259047691526
9TNyHeYcBJRCMJJzmPKZjEjKmvqTEWZX7phPCBaHaSz6 0.885215527083784
D676Df3CRZaWyvbWb4RGwzXmnnQSUdeiAjfvTKq5Wo63 0.22446862051474897
8cbnR416qGpfUxhBR4msuLH1zqcL7tZdU9E66RM6vsin 0
2S3iiCADVFYQdUSYYMBm47mYYZ7SYSkjuJiNZSE5RX39 0
9tzzat4TWQHNL1DdQEwr

In [14]:
res

{'CQikhhQ9STuD1S3A45DGvUAtCdSs5pw2YwJ8Mk6U9Fs7': 180.6459547686176,
 'ETv2idiPjun5RReJNSLBd2RUMcYSBMqjFQxhVpLeDqkm': 11.389008021627172,
 'GZYHrKmMPQLrmVGw7PRny7tP9wQZnFCC5vYfuaQiVTdh': 0,
 '24CoH2cYsEPv3EpigCUjsqs26n1XeDrtHug4kwX7UZGD': 89.30916431715201,
 '8vRaeK5hp4T3n8vyzfeHkHaUYqsXRrWBiAh3DDBn9dqe': 0,
 '2TGaPLxbFVgp2T2UycxEexbifxGeKbQGDVfiy2xZnoMN': 37.09273806905467,
 'E3Jwp134uzpA2MyCadYmeqWAxK8zDBmcNXTvSRrVK7EN': 11.768393815791308,
 'BxC131fG5TgGE1Wek7WtU4cTJcHWAs5cTmEmnx5BmQus': 3.3220402345281257,
 'BRNYKBf4wSfV8iZRa64fmjaerVUJjE38zR1Jmb8kQFkE': 9.466138087411348,
 'HdaigHQuEdsR6LsXZbgT37ohcbs1ZxEtSR2cudpWzZ2S': 0,
 '43o5vw2xsdC5qghLzrCrEB9z8Xz1JWFQU524nwAjFyKo': 0,
 'FPcL3sFutPQcV4hSvwBrhz19oJ6MNeGsaSMSYP1o4Rfh': 0,
 'CTgwTfirY2dtLbEBKYFe3T9KmeDLtf6QrHmtuAMwDTsP': 122.06814182254777,
 '7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN': 0,
 '775ANAGx3kyFQV6YHvQXmfA7gB6DFkyojsQZXVdF4vwr': 0.5113793686204104,
 'E6JjHtYpJkZqaYXa2WDccS39wABSkvkjzhwhe28rVKeZ': 0,
 '7g2UxLyBrHMFpGHKi

In [12]:
len(res)

1210

In [13]:
len(list(filter(lambda y: y[1] > 0.01, res.items())))

607

In [21]:
POOL = 13701


In [22]:
dff = pd.DataFrame(res.items(), columns=["userAddress", "vaultTokens"])

dff.vaultTokens.sum()


dff["usdc_send"] = dff.vaultTokens/dff.vaultTokens.sum()*POOL

write = dff.loc[dff.usdc_send > 5].sort_values("usdc_send")[::-1]

write.to_csv("luna_call_distribution.csv")

NameError: name 'res' is not defined

In [23]:
x = pd.read_csv("luna_call_distribution.csv")

x = x.append({"Unnamed: 0": 1, "userAddress": "7FkVHPNHTJ8Kz361kus3pVBwGEnoKGy3KfJJQ1QfHv7", "vaultTokens": 2000, "usdc_send": 0}, ignore_index=True)

x = x.drop_duplicates()

x["usdc_send"] = POOL*x.vaultTokens/x.vaultTokens.sum()

/var/folders/t5/lm3g0brx0dsf0ymt1ldc8xdr0000gp/T/ipykernel_88949/4061117991.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x = x.append({"Unnamed: 0": 1, "userAddress": "7FkVHPNHTJ8Kz361kus3pVBwGEnoKGy3KfJJQ1QfHv7", "vaultTokens": 2000, "usdc_send": 0}, ignore_index=True)


In [27]:
y = x.userAddress.values

z = x.usdc_send.values

np.savetxt('luna_call_addresses.log', y, fmt="%s")

np.savetxt('luna_call_amounts.log', z, fmt="%0.2f")